In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing

In [7]:
fluprint_export = pd.read_csv("C:/Users/Hanne/OneDrive - UGent/Documenten/1e master/1e semester/Large scale analysis of biomedical data/Practical sessions/fluprint_export.csv")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

In [8]:
# Look at the first 5 rows

In [9]:
fluprint_export.head()

,donor_id,study_id,gender,race,visit_id,visit_year,visit_day,visit_type_hai,visit_age,cmv_status,ebv_status,bmi,vaccine,geo_mean,d_geo_mean,vaccine_response,mesurment_id,assay,name,name_formatted,subset,units,data,statin_use,flu_vaccination_history,total_vaccines_received,vaccinated_1yr_prior,vaccine_type_1yr_prior,vaccinated_2yr_prior,vaccine_type_2yr_prior,vaccinated_3yr_prior,vaccine_type_3yr_prior,vaccinated_4yr_prior,vaccine_type_4yr_prior,vaccinated_5yr_prior,vaccine_type_5yr_prior,influenza_infection_history,influenza_hospitalization
0,813,15,Female,Caucasian,2937,2014,0,pre,23.0,0.0,0.0,NaN,4.0,380.55,1.0,0.0,371121,4,B cells,B_cells,CD14-CD33-/CD3-/CD19+CD20+,% of Parent,34.40,0.0,1.0,11.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,0,0
1,813,15,Female,Caucasian,2937,2014,0,pre,23.0,0.0,0.0,NaN,4.0,380.55,1.0,0.0,371122,4,basophils,basophils,CD123+HLADR-,% of Parent,1.45,0.0,1.0,11.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,0,0
2,813,15,Female,Caucasian,2937,2014,0,pre,23.0,0.0,0.0,NaN,4.0,380.55,1.0,0.0,371123,4,CD16+ monocytes,CD16_pos_monocytes,CD14+CD33+/CD16+,% of Parent,5.85,0.0,1.0,11.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,0,0
3,813,15,Female,Caucasian,2937,2014,0,pre,23.0,0.0,0.0,NaN,4.0,380.55,1.0,0.0,371124,4,CD16+CD14+ monocytes,CD16_pos_CD14_pos_monocytes,CD14+CD33+/CD14+CD16+,% of Parent,1.55,0.0,1.0,11.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,0,0
4,813,15,Female,Caucasian,2937,2014,0,pre,23.0,0.0,0.0,NaN,4.0,380.55,1.0,0.0,371125,4,CD16+CD14- monocytes,CD16_pos_CD14_neg_monocytes,CD14+CD33+/CD14-CD16+,% of Parent,4.46,0.0,1.0,11.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,0,0


In [10]:
# Make a dataframe that only contains the rows with measurements of cells ('data' = '% of Parent')

In [11]:
flu_cells = fluprint_export[fluprint_export.units == '% of Parent'] 

In [12]:
# Make the dataframe more compact and remove unnecessary columns

In [13]:
# Combine the indices of the columns to drop
columns_to_drop = list(range(1, 15)) + list(range(16, 19)) + list(range(20,22)) + list(range(23, len(fluprint_export.columns)))

# Drop the unnecessary columns
flu_cells_comp = flu_cells.drop(fluprint_export.columns[columns_to_drop], axis=1)
flu_cells_comp.head()

,donor_id,vaccine_response,name_formatted,data
0,813,0.0,B_cells,34.40
1,813,0.0,basophils,1.45
2,813,0.0,CD16_pos_monocytes,5.85
3,813,0.0,CD16_pos_CD14_pos_monocytes,1.55
4,813,0.0,CD16_pos_CD14_neg_monocytes,4.46


In [14]:
# Remove the rows with missing values for 'vaccine_response'

In [15]:
flu_cells_clean = flu_cells_comp.dropna(subset=['vaccine_response'])
print(flu_cells_clean)

        donor_id  vaccine_response                   name_formatted   data
0            813               0.0                          B_cells  34.40
1            813               0.0                        basophils   1.45
2            813               0.0               CD16_pos_monocytes   5.85
3            813               0.0      CD16_pos_CD14_pos_monocytes   1.55
4            813               0.0      CD16_pos_CD14_neg_monocytes   4.46
...          ...               ...                              ...    ...
155845         3               0.0  Th2_non_neg_TFH_CD8_pos_T_cells   9.71
155846         3               0.0          Th2_TFH_CD4_pos_T_cells  34.90
155847         3               0.0          Th2_TFH_CD8_pos_T_cells   1.84
155848         3               0.0             transitional_B_cells   6.51
155849         3               0.0                            Tregs   3.01

[25313 rows x 4 columns]


In [16]:
# Find duplicates of 'name_formatted' for one 'donor_id' and remove them so we can pivote the dataframe

In [17]:
# Find duplicates
duplicates = flu_cells_clean[flu_cells_clean.duplicated(subset=['donor_id', 'name_formatted'], keep=False)]
print(duplicates)

# Remove the duplicates
flu_cells_clean_unique = flu_cells_clean.drop_duplicates(subset=['donor_id', 'name_formatted'])
print(flu_cells_clean_unique)

       donor_id  vaccine_response       name_formatted  data
88836       325               0.0              B_cells  40.4
88837       325               0.0              B_cells  58.3
88838       325               0.0              B_cells  49.6
88847       325               0.0  CD3_neg_lymphocytes  15.2
88848       325               0.0  CD3_neg_lymphocytes  20.9
...         ...               ...                  ...   ...
98141       263               0.0              T_cells  83.9
98142       263               0.0              T_cells  80.8
98143       263               0.0              T_cells  82.4
98144       263               0.0              T_cells  82.5
98145       263               0.0              T_cells  82.1

[780 rows x 4 columns]
        donor_id  vaccine_response                   name_formatted   data
0            813               0.0                          B_cells  34.40
1            813               0.0                        basophils   1.45
2            813   

In [18]:
# Pivot the dataframe to create a new dataframe where each 'donor_id' is a row
# Each cell in 'name_formatted' becomes a column, and the corresponding 'data' values fill the cells

In [19]:
pivoted_df = flu_cells_clean_unique.pivot(index='donor_id', columns='name_formatted', values='data')

In [20]:
# Make a dataframe with 'donor_id' and 'vaccine_response'

In [21]:
vaccine_response = fluprint_export.drop_duplicates(subset=['donor_id', 'vaccine_response'])[['donor_id', 'vaccine_response']]
vaccine_response = vaccine_response.dropna()

In [22]:
# Merge 'pivoted_df' with 'vaccine_response' by the 'donor_id'

In [23]:
merged = vaccine_response.merge(pivoted_df, on='donor_id')

In [24]:
# Look at the correlation between the data of the other features and the 'vaccine_response'

In [25]:
# Calculate pairwise correlation coefficients between numerical columns in 'merged'
correlation = merged.corr()

# Sort correlations of 'vaccine_response' with other features
corr_response = correlation['vaccine_response'].sort_values(ascending=False)

# Display the most positively and negatively correlated features
print(corr_response.head())
print(corr_response.tail())


vaccine_response             1.000000
IL_neg_21_CD4_pos_T_cells    0.520964
Unstim_CD4_pos_T_cells       0.518184
PMA_Iono_CD4_pos_T_cells     0.516098
LPS_CD4_pos_T_cells          0.506828
Name: vaccine_response, dtype: float64
IL_neg_7_CD8_pos_T_cells    -0.593862
IL_neg_21_CD8_pos_T_cells   -0.594105
PMA_Iono_CD8_pos_T_cells    -0.594758
LPS_CD8_pos_T_cells         -0.599727
Unstim_CD8_pos_T_cells      -0.605982
Name: vaccine_response, dtype: float64


In [26]:
# Look how many donors have data for every cell type

In [30]:
non_na_counts = merged.notna().sum()
print(non_na_counts)

donor_id                      304
vaccine_response              304
B_cells                       294
CD161_neg_CD45RA_pos_Tregs    294
CD161_pos_CD45RA_neg_Tregs    294
                             ... 
naive_CD4_pos_T_cells         294
naive_CD8_pos_T_cells         294
pDCs                          164
plasmablasts                  294
transitional_B_cells          294
Length: 413, dtype: int64


In [31]:
# Combine 'corr_response' and 'non_na_counts' into a single dataframe

In [32]:
combined_df = pd.DataFrame({
    'Corr_response': corr_response,
    'Non_Na_Counts': non_na_counts
})

# Sort by correlation values 
combined_df = combined_df.sort_values(by='Corr_response', ascending=False)
print(combined_df)

                           Corr_response  Non_Na_Counts
vaccine_response                1.000000            304
IL_neg_21_CD4_pos_T_cells       0.520964             18
Unstim_CD4_pos_T_cells          0.518184             18
PMA_Iono_CD4_pos_T_cells        0.516098             18
LPS_CD4_pos_T_cells             0.506828             18
...                                  ...            ...
IL_neg_7_CD8_pos_T_cells       -0.593862             18
IL_neg_21_CD8_pos_T_cells      -0.594105             18
PMA_Iono_CD8_pos_T_cells       -0.594758             18
LPS_CD8_pos_T_cells            -0.599727             18
Unstim_CD8_pos_T_cells         -0.605982             18

[413 rows x 2 columns]


In [33]:
# Filter the features with more than 290 measured values and with a absolute correlation to 'vaccine_response' of more than 0.1 
# Or filter the feature with 304 values to make sure that 'donor_id' is kept in the dataframe

In [35]:
# Apply the filter conditions 
filtered_df = combined_df[((combined_df['Non_Na_Counts'] > 290) & (combined_df['Corr_response'].abs() > 0.1)) | (combined_df['Non_Na_Counts'] == 304)] 

# Display all rows of the DataFrame 
with pd.option_context('display.max_rows', None): 
    print(filtered_df)   

                                    Corr_response  Non_Na_Counts
vaccine_response                         1.000000            304
CD161_pos_CD45RA_pos_Tregs               0.263836            294
CD85j_pos_CD4_pos_T_cells                0.174133            294
T_cells                                  0.139733            294
CD161_pos_CD45RA_neg_Tregs               0.120417            294
donor_id                                -0.015260            304
central_memory_CD8_pos_T_cells          -0.101233            294
plasmablasts                            -0.106176            294
HLADR_pos_CD38_pos_CD8_pos_T_cells      -0.109893            294
HLADR_neg_CD38_pos_CD8_pos_T_cells      -0.117639            293
HLADR_pos_CD38_pos_CD4_pos_T_cells      -0.131227            294
HLADR_neg_CD38_pos_CD4_pos_T_cells      -0.141555            294


In [37]:
# Extract names from the index and use them to filter these columns out of the 'merged' dataframe

In [41]:
# Extract names from the index
relevant_cells = filtered_df.index.tolist() 

# Filter the pivoted dataframe using these names
df = merged[relevant_cells] 

In [42]:
# Determine the distribution of 'vaccine_response' in the dataframe to make sure that there are enough rows for both possible outcomes

In [55]:
df['vaccine_response'].value_counts()

vaccine_response
0.0    215
1.0     89
Name: count, dtype: int64

In [57]:
# Set 'vaccine_response' as the prediction target

In [46]:
y = df.vaccine_response

In [47]:
# Print the columns names of 'df'

In [48]:
print(df.columns)

Index(['vaccine_response', 'CD161_pos_CD45RA_pos_Tregs',
       'CD85j_pos_CD4_pos_T_cells', 'T_cells', 'CD161_pos_CD45RA_neg_Tregs',
       'donor_id', 'central_memory_CD8_pos_T_cells', 'plasmablasts',
       'HLADR_pos_CD38_pos_CD8_pos_T_cells',
       'HLADR_neg_CD38_pos_CD8_pos_T_cells',
       'HLADR_pos_CD38_pos_CD4_pos_T_cells',
       'HLADR_neg_CD38_pos_CD4_pos_T_cells'],
      dtype='object')


In [ ]:
# Keep the column names with cell information and set them as the predictive features

In [50]:
flu_features = ['CD161_pos_CD45RA_pos_Tregs',
       'CD85j_pos_CD4_pos_T_cells', 'T_cells', 'CD161_pos_CD45RA_neg_Tregs', 'central_memory_CD8_pos_T_cells', 'plasmablasts',
       'HLADR_pos_CD38_pos_CD8_pos_T_cells',
       'HLADR_neg_CD38_pos_CD8_pos_T_cells',
       'HLADR_pos_CD38_pos_CD4_pos_T_cells',
       'HLADR_neg_CD38_pos_CD4_pos_T_cells']
X = df[flu_features]

In [ ]:
# Split the dataframe in a train and test dataset en use 'DecisionTreeRegressor' as model
# Also look at the 'mean_absolute_error' to determine the mean deviation from the true 'vaccine_response'

In [58]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Split the data into training and validation datasets
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=0)

# Define and train the 'DecisionTreeClassifier'
flu_model = DecisionTreeClassifier(random_state=0)
flu_model.fit(train_X, train_y)

# Make predictions on the validation dataset
val_predictions = flu_model.predict(val_X)

# Evaluate the model using accuracy score
print(accuracy_score(val_y, val_predictions))

0.631578947368421


In [59]:
# print the top few validation predictions
print(flu_model.predict(val_X.head()))
# print the top few actual vaccine responses from validation data
print(val_y.head())

[1. 0. 0. 0. 0.]
66     1.0
156    1.0
247    0.0
267    0.0
145    0.0
Name: vaccine_response, dtype: float64
